In [51]:
import os

In [68]:
%pwd

'D:\\Proj_files\\Project_Machine_Learning\\Completed projects\\Text Summariation'

In [69]:
os.chdir("../")
os.chdir('D:\Proj_files\Project_Machine_Learning\Completed projects\Text Summariation\Text_Summarizer_Project_01')

In [70]:
%pwd

'D:\\Proj_files\\Project_Machine_Learning\\Completed projects\\Text Summariation\\Text_Summarizer_Project_01'

In [80]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories
from pathlib import Path, PurePath

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

print(CONFIG_FILE_PATH)
A = read_yaml(CONFIG_FILE_PATH)
print(A)
print(A.artifacts_root)


config\config.yaml
[2023-06-12 11:15:26,130: INFO: common: yaml file: config\config.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/srivanoo21/Text_Summarizer_Project_01/raw/main/summarizer-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
artifacts


In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path



In [76]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [86]:
class ConfigurationManager:
    
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config
            
            

In [9]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [30]:
class DataIngestion:

    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size{get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)           

In [88]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e
    

[2023-06-12 11:19:22,273: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-12 11:19:22,279: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-12 11:19:22,282: INFO: common: created directory at : artifacts]
[2023-06-12 11:19:22,285: INFO: common: created directory at : artifacts/data_ingestion]
[2023-06-12 11:19:29,108: INFO: 1326566912: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FF42:0806:207D09:312401:6486B1E3
Accept-Ranges: bytes
Date: Mon, 12 Jun 2023 05:49:24 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830048-CCU
X-Cache: 